## For track information 

In [1]:
import pandas as pd

df = pd.read_csv("cleaned_data/seeds.csv")

In [2]:
# place SQL results in a DataFrame - method 2: pandas read_sql_query()
import sqlite3 as sql
import pandas as pd
connection = sql.Connection("raw_data/track_metadata.db")
query = 'SELECT * FROM songs'
meta = pd.read_sql_query(query, con=connection)

In [3]:
meta = meta[meta.track_id.isin(df.track_id)].reset_index(drop=True)

# drop last three columns
meta = meta.iloc[:, :-3]

meta.head()

,track_id,title,song_id,release,artist_id,artist_mbid,artist_name,duration,artist_familiarity,artist_hotttnesss,year
0,TRMMMYQ128F932D901,Silent Night,SOQMMHC12AB0180CB8,Monster Ballads X-Mas,ARYZTJS1187B98C555,357ff05d-848a-44cf-b608-cb34b5701ae5,Faster Pussy cat,252.05506,0.649822,0.394032,2003
1,TRMMMCH128F425532C,Si Vos Querés,SOBNYVR12A8C13558C,De Culo,ARNWYLR1187B9B2F9C,12be7648-7094-495f-90e6-df4189d68615,Yerba Brava,145.05751,0.448501,0.372349,2003
2,TRMMMNS128F93548E1,L'antarctique,SOYGNWH12AB018191E,Des cobras des tarentules,AR59BSJ1187FB4474F,891fccfc-24c1-4bfd-bf49-c736e59e443f,3 Gars Su'l Sofa,68.96281,0.555014,0.352949,2007
3,TRMMMXI128F4285A3F,N Gana,SOGPCJI12A8C13CCA0,Afropea 3 - Telling Stories To The Sea,ARBAMQB1187FB3C650,0bb5e108-b41d-46cd-969e-69d34d1acdfe,Waldemar Bastos,273.18812,0.543690,0.373679,0
4,TRMMMKI128F931D80D,006,SOSDCFG12AB0184647,Lena 20 År,ARSB5591187B99A848,fba3e876-68f1-4a1f-99d9-c604480202ba,Lena Philipsson,262.26893,0.529819,0.410229,1998


In [4]:
# save metadata
meta.to_csv("cleaned_data/tracks_metadata.csv", index=False)

## For Artists

In [5]:
# artist tags database

connection = sql.Connection("raw_data/artist_term.db")
query = 'SELECT * FROM artist_mbtag'
artist = pd.read_sql_query(query, con=connection)

In [6]:
artist.head()

,artist_id,mbtag
0,AR002UA1187B9A637D,uk
1,AR002UA1187B9A637D,rock
2,AR002UA1187B9A637D,garage rock
3,AR006821187FB5192B,bass
4,AR00A6H1187FB5402A,detroit


In [7]:
# combine rows into lists

artist = artist.groupby("artist_id")['mbtag'].apply(list).reset_index(name='tags')

In [8]:
artist2 = pd.read_csv("raw_data/artist_location.txt", header=None,
                   sep='<SEP>', names=['artist_id','lng','lat','artist_name','place'])

/Users/ginger/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [9]:
artist = artist.merge(artist2, how='left', on='artist_id')
# select artist in our listening history

artist = artist[artist.artist_id.isin(meta.artist_id)].reset_index(drop=True)
artist.drop(columns='artist_name', axis=1, inplace=True)

In [10]:
artist.head(5)

,artist_id,tags,lng,lat,place
0,AR00B1I1187FB433EB,"[european, swedish, classic pop and rock]",NaN,NaN,NaN
1,AR00FVC1187FB5BE3E,"[german, berlin]",25.67084,-100.30953,"Monterrey, NL, México"
2,AR00IKC1187FB37BD5,"[uk, scotland, edinburgh, punk, anarcho]",NaN,NaN,NaN
3,AR00M9H1187B9B59CA,[virgin islands reggae],47.67885,-122.20724,"Kirkland, WA"
4,AR00P3M1187FB5428C,[british],NaN,NaN,NaN


In [11]:
artist.to_csv("cleaned_data/artist.csv", index=False)

## Lyrics 

https://github.com/tbertinmahieux/MSongsDB/blob/master/Tasks_Demos/Lyrics/README.txt

It is worth discussion for better representation format.

In [12]:
# place SQL results in a DataFrame - method 2: pandas read_sql_query()
connection = sql.Connection("raw_data/lyrics.db")
query = 'SELECT * FROM words'
words = pd.read_sql_query(query, con=connection)

In [14]:
words.to_csv("cleaned_data/word_list.csv")

In [15]:
query = 'SELECT * FROM lyrics'
lyrics = pd.read_sql_query(query, con=connection)

In [17]:
lyrics.head(10)

,track_id,mxm_tid,word,count,is_test
0,TRAAAAV128F421A322,4623710,i,6,0
1,TRAAAAV128F421A322,4623710,the,4,0
2,TRAAAAV128F421A322,4623710,you,2,0
3,TRAAAAV128F421A322,4623710,to,2,0
4,TRAAAAV128F421A322,4623710,and,5,0
5,TRAAAAV128F421A322,4623710,a,3,0
6,TRAAAAV128F421A322,4623710,me,1,0
7,TRAAAAV128F421A322,4623710,it,1,0
8,TRAAAAV128F421A322,4623710,my,1,0
9,TRAAAAV128F421A322,4623710,is,2,0


In [18]:
lyrics.to_csv("cleaned_data/lyrics.csv")

## Artists Similarity

In [1]:
import sqlite3 as sql
import pandas as pd
connection = sql.Connection(r"E:\MSBA\Courses\Spring\Big Data Analytics\Project\Phase2\Data\artist_similarity.db")

In [4]:
query = 'SELECT * FROM artists'
artists = pd.read_sql_query(query, con=connection)

In [5]:
artists

,artist_id
0,AR002UA1187B9A637D
1,AR003FB1187B994355
2,AR006821187FB5192B
3,AR009211187B989185
4,AR009SZ1187B9A73F4
...,...
44740,ARZZXT51187FB4627E
44741,ARZZYRB1187B99D0B6
44742,ARZZYRH11C8A416A12
44743,ARZZZAI124207819C9


In [6]:
query2 = 'SELECT * FROM similarity'
similarity = pd.read_sql_query(query2, con=connection)

In [7]:
similarity

,target,similar
0,AR002UA1187B9A637D,ARQDOR81187FB3B06C
1,AR002UA1187B9A637D,AROHMXJ1187B989023
2,AR002UA1187B9A637D,ARAGWVR1187B9B749B
3,AR002UA1187B9A637D,AREQVWS1241B9CC0A4
4,AR002UA1187B9A637D,ARHBE351187FB3B0CD
...,...,...
2201911,ARZZZKG1271F573BC4,ARBYTEZ1187FB42EC2
2201912,ARZZZKG1271F573BC4,AR3B4IC1187B98C8B6
2201913,ARZZZKG1271F573BC4,ARM9KFI1187B9B356C
2201914,ARZZZKG1271F573BC4,ARF5QVT1187B9946B9


In [17]:
# Group similar artists into list for an artist
similarity_new = similarity.groupby('target')['similar'].apply(list).reset_index(name='new')

In [18]:
similarity_new

,target,new
0,AR002UA1187B9A637D,"[ARQDOR81187FB3B06C, AROHMXJ1187B989023, ARAGW..."
1,AR003FB1187B994355,"[ARYACSL1187FB51611, ARYLCCQ1187B999F4B, AR783..."
2,AR006821187FB5192B,"[ARW25O21187B991492, ARQKS2U1187FB4CFBA, ARRKD..."
3,AR009211187B989185,"[ARJRM4M1187B9B4462, ARHINI31187B995C1D, ARI0P..."
4,AR009SZ1187B9A73F4,"[ARY8CFI1187B98D5E3, ARO03MT1187B9A8F2D, AR2NW..."
...,...,...
44455,ARZZXJY1187B99E2BB,"[AREJ5K11187B993F5F, AR5AXVN1187B9A2761, ARXXX..."
44456,ARZZXT51187FB4627E,"[ARVIDW81187FB5AAC3, ARC4AJX1187FB3C6BC, ARBZ6..."
44457,ARZZYRB1187B99D0B6,"[AR4XV7Y1187FB41004, ARLELZZ1187B993920, ARR0C..."
44458,ARZZYRH11C8A416A12,"[ARLSFWF12086C152F4, AR0VU8Y11C8A422C79, ARS7P..."


In [19]:
similarity_new.to_csv(r'E:\MSBA\Courses\Spring\Big Data Analytics\Project\Phase2\Data\artist_similarity.csv', index=False)